# 머신러닝 기술을 활용한 ​​STEAM 리뷰 키워드 분류​

## 프로젝트 배경

- 스팀은 방대한 양의 리뷰 데이터를 보유하고 있음에도 현재로서는 그 내용에 대해 분석하는 시스템이 존재하지 않는다. 사용자들은 어떠한 부분에 대해 긍정적으로 평가하는지, 어떤 문제점이 있었는지 등을 파악하기 위해 상당히 많은 수의 리뷰를 읽어보아야 한다. 
- 간혹 전쟁이나 정치 등의 지엽적인 이유로 좋지 않은 평을 받게 되는 게임들이 존재하며, 이런 경우 사용자가 게임을 파악하는 데에 장애가 되곤 하므로 리뷰의 내용을 분석하는 기능이 필요하다. 
- 요즘 시대에 게임은 단순히 게임 플레이만이 중요한 판단 요소가 되는 것이 아닌, 스토리텔링을 비롯한 기타 콘텐츠들이 판단 척도로 포함되고 있으므로 사용자에게 이러한 부분에 대한 정보를 제공할 필요가 있다.

## 진행 내용

각 단계별로 진행상황을 확인하기 위해 결과를 출력하는 과정을 삽입했고, 결과적으로는 게임별로 분류를 진행해야 하므로 가장 하단에서는 한번에 모든 과정을 진행할 수 있도록 각 단계를 함수로 묶어 코드를 작성하였다.

1. 각 게임의 리뷰 데이터를 추출한다.
2. HDBSCAN을 활용하여 리뷰를 주제별로 클러스터링 한다.
3. 표제어를 추출하고 불용어를 제거한다.
4. LDA를 활용하여 토픽 모델링을 하고 클러스터의 라벨을 바이그램으로 추출한다.
5. 각 게임의 긍정, 부정적 리뷰의 분석 결과를 정리하여 볼 수 있도록 나타낸다.

**! 런타임 유형 - 하드웨어 가속기를 GPU로 설정하고 실행해야 합니다 !**

# 1. 리뷰 데이터를 추출

## 필요한 패키지 설치

In [1]:
import numpy as np 
import pandas as pd 
import random as rn
import re
import nltk
import os

In [2]:
# 클러스터링에 필요한 패키지
!pip install umap-learn
!pip install hdbscan
from umap import UMAP
from hdbscan import HDBSCAN

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 7.9 MB/s 
     |████████████████████████████████| 1.1 MB 54.1 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=6c359a09d462f9214e6ff292dc6fc3d352d37c82983d020689269cc5b7ddf6ea
  Stored in directory: /root/.cache/pip/wheels/a9/3a/67/06a8950e053725912e6a8c42c4a3a241410f6487b8402542ea
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55513 sha256=35f24110486db04e0d4001903fb8163fe80db8b079362d9d8b02984873b0af21
  Stored in directory: /root/.cache/pip/wheels/1c/63/3a/29954bca1a27ba100ed8c27973a78cb71b43dc67aed62e80c3
Successfully built umap-learn pynndescent
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.2 MB 20.1 MB/s 
  Installing build dependencies ... done
  Getting requi

In [3]:
# 문장 임베딩에 필요한 패키지
nltk.download('punkt')
!pip install -U -q sentence-transformers
from sentence_transformers import SentenceTransformer
import tensorflow as tf

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


     |████████████████████████████████| 85 kB 4.8 MB/s 
     |████████████████████████████████| 5.8 MB 64.3 MB/s 
     |████████████████████████████████| 1.3 MB 67.0 MB/s 
     |████████████████████████████████| 182 kB 77.8 MB/s 
     |████████████████████████████████| 7.6 MB 66.7 MB/s 


In [4]:
# 토픽 모델링에 필요한 패키지
import string
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer= WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.corpus import wordnet

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## 데이터셋 불러오기

kaggle에 업로드 되어 있는 데이터를 사용할 것이므로, kaggle api를 사용한다. <br/>
따라서 먼저 kaggle.json 파일을 업로드해주어야 한다. <br/>
데이터셋은 다음 링크의 csv 파일을 사용했다. <br/>
https://www.kaggle.com/datasets/andrewmvd/steam-reviews

In [5]:
!pip show kaggle

Name: kaggle
Version: 1.5.12
Summary: Kaggle API
Home-page: https://github.com/Kaggle/kaggle-api
Author: Kaggle
Author-email: support@kaggle.com
License: Apache 2.0
Location: /usr/local/lib/python3.8/dist-packages
Requires: python-dateutil, tqdm, urllib3, certifi, requests, python-slugify, six
Required-by: 


In [6]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls -la  ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!ls -la  ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 64 Dec 17 07:16 /root/.kaggle/kaggle.json
-rw------- 1 root root 64 Dec 17 07:16 /root/.kaggle/kaggle.json


In [7]:
!kaggle datasets download -d andrewmvd/steam-reviews

100% 685M/685M [00:35<00:00, 22.3MB/s]
100% 685M/685M [00:36<00:00, 20.0MB/s]


In [8]:
!unzip steam-reviews.zip

Archive:  steam-reviews.zip
  inflating: dataset.csv             


df 변수에 내용을 저장하고 형태를 살펴보았을 때, app_name에 게임 이름이, review_text에 리뷰 내용이 담겨있고 review_score에 긍정, 부정 여부가 저장되어있음을 알 수 있다.

In [9]:
df = pd.read_csv('./dataset.csv')
print(df.shape)

(6417106, 5)


In [10]:
df.head()

,app_id,app_name,review_text,review_score,review_votes
0,10,Counter-Strike,Ruined my life.,1,0
1,10,Counter-Strike,This will be more of a ''my experience with th...,1,1
2,10,Counter-Strike,This game saved my virginity.,1,0
3,10,Counter-Strike,• Do you like original games? • Do you like ga...,1,0
4,10,Counter-Strike,"Easy to learn, hard to master.",1,1


In [11]:
# 게임 제목을 기준으로 리뷰 개수를 카운트
count_df = pd.DataFrame(df.app_name.value_counts())
count_df.head()

,app_name
PAYDAY 2,88973
DayZ,88850
Terraria,84828
Rust,77037
Dota 2,73541


## 전처리 작업

In [12]:
# -1로 표기된 부정적인 리뷰는 0으로 바꿈
df["review_score"] = np.where(df["review_score"]==-1, 0, df["review_score"])
# 내용이 없는 리뷰들은 삭제(Early Access Review라고 적힌 리뷰 역시 내용이 없으므로 삭제)
df = df[df.review_text != "Early Access Review"]
df = df[~df.review_text.isin(['nan'])]
# 중복이 있다면 삭제
df.drop_duplicates(['review_text', 'review_score'], inplace = True)

print(df.shape)

(4476235, 5)


In [13]:
count_df = pd.DataFrame(df.app_name.value_counts())
count_df.head()

,app_name
Terraria,77390
PAYDAY 2,61891
Undertale,47172
Dota 2,46640
Warframe,43477


전처리를 완료했을 때 리뷰 개수가 많은 상위 3개의 게임을 대상으로 리뷰 분석을 진행할 것이므로 각 리뷰들을 분리하여 저장한다.

In [14]:
terraria = df[df['app_name'] == 'Terraria']
payday = df[df['app_name'] == 'PAYDAY 2']
undertale = df[df['app_name'] == 'Undertale']

In [15]:
# 긍정적/부정적 리뷰 역시 분리
pos_terraria = terraria[terraria.review_score == 1]
neg_terraria = terraria[terraria.review_score == 0]

pos_payday = payday[payday.review_score == 1]
neg_payday = payday[payday.review_score == 0]

pos_undertale = undertale[undertale.review_score == 1]
neg_undertale = undertale[undertale.review_score == 0]

In [16]:
print(pos_terraria.shape, neg_terraria.shape)

(75176, 5) (2214, 5)


In [17]:
print(pos_payday.shape, neg_payday.shape)

(42445, 5) (19446, 5)


In [18]:
print(pos_undertale.shape, neg_undertale.shape)

(45287, 5) (1885, 5)


# 2. 리뷰를 주제별로 클러스터링

In [19]:
# 먼저 텍스트 전처리를 실행함
def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[♥]+','', text)
    return text

In [20]:
# 리뷰 토큰화
def tokenize(dataset):
  dataset = dataset.review_text.apply(clean_text)
  data_list = dataset.tolist()

  data_sents = []
  for r in data_list:
    for sent in nltk.sent_tokenize(r):
      # 문장 길이가 지나치게 짧은 경우 제거함
      if len(sent.split())>4:
        data_sents.append(sent)
  return data_sents

**SentenceTransformer: 텍스트 임베딩을 위한 pre-trained 모델.** <br/>
다양한 transformer 모델 중 가장 속도와 퍼포먼스 밸런스가 좋다고 생각되어 all-MiniLM-L6-v2 모델을 선택하였다. <br />
참고자료: https://www.sbert.net/docs/pretrained_models.html

In [21]:
def embed(testset):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(testset)
    return embeddings

**UMAP: 차원 축소 알고리즘** <br/>
**HDBSCAN: 밀도 기반의 클러스터링인 DBSCAN의 확장 버전으로 클러스터별로 다른 밀도를 가질 수 있어 보다 높은 유연성을 가진다.** <br />
transformer를 사용하여 임베딩된 결과물은 고차원의 벡터로 나타내어지므로, UMAP을 활용하여 차원을 축소시켜야 클러스터링이 효율적으로 가능하다.

In [22]:
def generate_clusters(sent_list):
  umap_embeddings = UMAP(n_neighbors=15, 
                      n_components=10, 
                      min_dist=0.0, 
                      metric='cosine', 
                      random_state=42
                    ).fit_transform(sent_list)

# 클러스터가 지나치게 세분화되지 않도록 최소 클러스터 사이즈를 50으로 지정
  clusters = HDBSCAN(min_cluster_size = 50, 
                      min_samples = None,
                      metric='euclidean', 
                      gen_min_span_tree=True,
                      cluster_selection_method='eom').fit(umap_embeddings)

  return clusters

In [23]:
def clustering(dataset):
  tokenized_data = tokenize(dataset)
  embeded_data = embed(tokenized_data)
  cluster = generate_clusters(embeded_data)
  data_clustered = pd.DataFrame(data = list(zip(tokenized_data, cluster.labels_)), 
                              columns = ['text', 'label'])
  return data_clustered

## 클러스터링 실행하고 결과를 확인

In [24]:
pos_terraria_cluster = clustering(pos_terraria)
pos_terraria_cluster.head()

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

,text,label
0,you can be a werewolf riding a unicorn while s...,-1
1,"you can build teleporters, find a hair dresser...",-1
2,you can take on the lord of the moon using a y...,-1
3,you can buy a music box from a wizard and go r...,378
4,it would seem the only thing minecraft has ove...,62


# 3. 표제어를 추출하고 불용어를 제거

**stoplist: nltk가 정의한 불용어 리스트** <br/>
stoplist를 활용하여 무의미한 단어 토큰은 제외한다. 이때 게임 제목과 game, play 등 모든 게임에서 공통적으로 나타나는 단어를 stoplist에 포함시켰다.

In [25]:
stoplist= stopwords.words('english')

In [26]:
common_terms= ["game", "play","people","terraria","payday", "undertale"]
stoplist = stoplist+ common_terms

In [27]:
def get_tag(tag):
    if tag.startswith('N') or tag.startswith('J'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV

In [28]:
def text_process(text):
    punctuation= list(string.punctuation)
    doc_tokens= nltk.word_tokenize(text)
    word_tokens= [word.lower() for word in doc_tokens if not (word in punctuation or len(word)<=3)]
   
    pos_tags=nltk.pos_tag(word_tokens)

    # nltk.pos_tag로 반환받는 형용사는 J로 시작하기 때문에 lemmatize 전에 a로 변환
    j_to_a = lambda x : x if x != 'J' else 'a'

    # stoplist에 포함되지 않는 토큰만 필터링하여 표제어를 추출
    words = [(word, j_to_a(tag)) for (word, tag) in pos_tags if tag[0] in ['V', 'J', 'N', 'R']]
    doc_words=[wordnet_lemmatizer.lemmatize(word, pos=get_tag(tag)) for word, tag in words]
    doc_words = [word for word in doc_words if word not in stoplist]
    return doc_words

## 텍스트 전처리를 실행하고 결과를 확인

In [29]:
pos_terraria_cluster['text_processed'] = pos_terraria_cluster['text'].apply(text_process)
pos_terraria_cluster.head()

,text,label,text_processed
0,you can be a werewolf riding a unicorn while s...,-1,"[werewolf, rid, unicorn, shoot, rainbow]"
1,"you can build teleporters, find a hair dresser...",-1,"[build, teleporters, find, hair, dresser, spid..."
2,you can take on the lord of the moon using a y...,-1,"[take, lord, moon, use, yoyo, summon, sharknad..."
3,you can buy a music box from a wizard and go r...,378,"[music, wizard, record, music, base, want, bui..."
4,it would seem the only thing minecraft has ove...,62,"[seem, thing, minecraft, dimension]"


# 4. 토픽 모델링을 통한 라벨링

**LDA: 단어 확률 분포에 기반하여 문장의 주제를 찾아내는 알고리즘** <br />
gensim에서 제공하는 phrases를 활용하여 단어 조합의 빈도수를 알아내고 해당 조합을 연결하여 라벨을 추출했다. 

In [30]:
import gensim
from collections import Counter
from gensim.models import Word2Vec

In [31]:
def extract_labels(docs):
  bi_phrases = gensim.models.Phrases(docs, min_count=5, threshold=20, delimiter=b'_')
  bigram_model = gensim.models.phrases.Phraser(bi_phrases)

  bigram_counter= Counter()
  for key in bi_phrases.vocab.keys():
    if key.decode() not in stoplist: 
      if len(str(key).split('_'))>1:
          bigram_counter[key]+=bi_phrases.vocab[key]

  # 빈도수가 가장 많은 두개의 바이그램을 반환
  label_words = bigram_counter.most_common(2)  
  label_words = [key.decode() for key, counts in label_words]

  return label_words

In [32]:
# 클러스터 라벨을 입력하면 해당 클러스터의 리뷰들을 반환하는 함수
def get_group(df, category_col, category):
    single_category = df[df[category_col]==category].reset_index(drop=True)
    return single_category 

In [33]:
def apply_and_summarize_labels(df, category_col):
    numerical_labels = df[category_col].unique()
    
    # 라벨(클러스터)별로 리스트를 만들고 리스트를 활용하여 라벨을 추출
    label_dict = {}
    for label in numerical_labels:
        current_category = list(get_group(df, category_col, label)['text_processed'])
        label_dict[label] = extract_labels(current_category)

    # 클러스터가 큰 순서(리뷰가 많은 순서)로 묶어 정렬
    summary_df = (df.groupby(category_col)['text'].count()
                    .reset_index()
                    .rename(columns={'text':'count'})
                    .sort_values('count', ascending=False))

    # 바이그램으로 제작한 라벨을 dataFrame에 추가
    summary_df['label'] = summary_df.apply(lambda x: label_dict[x[category_col]], axis = 1)
    
    return summary_df

## 토픽 모델링을 실행하고 결과를 확인

In [34]:
pos_terraria_summary = apply_and_summarize_labels(pos_terraria_cluster, 'label')
pos_terraria_summary.head(10)

,label,count
0,"[it.bye_it.bye, cool_cool]",78136
161,"[update_come, update_add]",5448
378,"[many_boss, fight_boss]",5090
285,"[many_hour, hour_still]",4532
43,"[terrarium_minecraft, minecraft_terrarium]",3259
10,"[best_sandbox, sandbox_ever]",3144
68,"[best_ever, probably_best]",1844
294,"[really_good, pretty_good]",1524
97,"[best_steam, first_steam]",1447
332,"[randomly_generate, generate_world]",1420


# 5. 각 게임의 리뷰를 분석

In [35]:
pos_terraria_cluster = clustering(pos_terraria)
pos_terraria_cluster['text_processed'] = pos_terraria_cluster['text'].apply(text_process)
pos_terraria_summary = apply_and_summarize_labels(pos_terraria_cluster, 'label')
pos_terraria_summary.head()

,label,count
0,"[it.bye_it.bye, cool_cool]",78136
161,"[update_come, update_add]",5448
378,"[many_boss, fight_boss]",5090
285,"[many_hour, hour_still]",4532
43,"[terrarium_minecraft, minecraft_terrarium]",3259


In [36]:
neg_terraria_cluster = clustering(neg_terraria)
neg_terraria_cluster['text_processed'] = neg_terraria_cluster['text'].apply(text_process)
neg_terraria_summary = apply_and_summarize_labels(neg_terraria_cluster, 'label')
neg_terraria_summary.head()

,label,count
1,"[waste_money, single_player]",4787
2,"[minecraft_well, minecraft_minecraft]",405
5,"[love_terrarium, make_terrarium]",295
3,"[starbound_instead, starbound_much]",73
0,"[stop_work, sandbox_survival]",69


In [37]:
terraria_review = pd.DataFrame({'positive':list(pos_terraria_summary['label'][:5]), 'negative':list(neg_terraria_summary['label'][:5])})
terraria_review

,positive,negative
0,"[it.bye_it.bye, cool_cool]","[waste_money, single_player]"
1,"[update_come, update_add]","[minecraft_well, minecraft_minecraft]"
2,"[many_boss, fight_boss]","[love_terrarium, make_terrarium]"
3,"[many_hour, hour_still]","[starbound_instead, starbound_much]"
4,"[terrarium_minecraft, minecraft_terrarium]","[stop_work, sandbox_survival]"


In [38]:
pos_payday_cluster = clustering(pos_payday)
pos_payday_cluster['text_processed'] = pos_payday_cluster['text'].apply(text_process)
pos_payday_summary = apply_and_summarize_labels(pos_payday_cluster, 'label')
pos_payday_summary.head()

,label,count
0,"[rob_bank, skill_tree]",36604
18,"[weapon_pack, many_dlcs]",7807
30,"[complete_heist, different_heist]",3514
120,"[stealth_mission, stealth_loud]",3016
92,"[complete_mission, variety_mission]",1473


In [39]:
neg_payday_cluster = clustering(neg_payday)
neg_payday_cluster['text_processed'] = neg_payday_cluster['text'].apply(text_process)
neg_payday_summary = apply_and_summarize_labels(neg_payday_cluster, 'label')
neg_payday_summary.head()

,label,count
0,"[weapon_skin, stat_boost]",26964
80,"[dlcs_dlcs, full_price]",5870
24,"[add_microtransactions, microtransactions_micr...",3498
14,"[shoot_shoot, rob_bank]",2761
55,"[micro_transaction, add_microtransactions]",1642


In [40]:
payday_review = pd.DataFrame({'positive':list(pos_payday_summary['label'][:5]), 'negative':list(neg_payday_summary['label'][:5])})
payday_review

,positive,negative
0,"[rob_bank, skill_tree]","[weapon_skin, stat_boost]"
1,"[weapon_pack, many_dlcs]","[dlcs_dlcs, full_price]"
2,"[complete_heist, different_heist]","[add_microtransactions, microtransactions_micr..."
3,"[stealth_mission, stealth_loud]","[shoot_shoot, rob_bank]"
4,"[complete_mission, variety_mission]","[micro_transaction, add_microtransactions]"


In [41]:
pos_undertale_cluster = clustering(pos_undertale)
pos_undertale_cluster['text_processed'] = pos_undertale_cluster['text'].apply(text_process)
pos_undertale_summary = apply_and_summarize_labels(pos_undertale_cluster, 'label')
pos_undertale_summary.head()

,label,count
0,"[make_feel, long_time]",59794
11,"[great_soundtrack, amaze_soundtrack]",9600
37,"[best_ever, favorite_time]",4438
91,"[multiple_ending, different_ending]",3001
21,"[spoil_anything, avoid_spoiler]",2238


In [42]:
neg_undertale_cluster = clustering(neg_undertale)
neg_undertale_cluster['text_processed'] = neg_undertale_cluster['text'].apply(text_process)
neg_undertale_summary = apply_and_summarize_labels(neg_undertale_cluster, 'label')
neg_undertale_summary.head()

,label,count
0,"[suck_liek, liek_agrithis]",3988
1,"[negative_review, bullet_hell]",535
5,"[music_great, good_soundtrack]",383
17,"[negative_review, positive_review]",363
12,"[suks_suks, night_freddy]",357


In [43]:
undertale_review = pd.DataFrame({'positive':list(pos_undertale_summary['label'][:5]), 'negative':list(neg_undertale_summary['label'][:5])})
undertale_review

,positive,negative
0,"[make_feel, long_time]","[suck_liek, liek_agrithis]"
1,"[great_soundtrack, amaze_soundtrack]","[negative_review, bullet_hell]"
2,"[best_ever, favorite_time]","[music_great, good_soundtrack]"
3,"[multiple_ending, different_ending]","[negative_review, positive_review]"
4,"[spoil_anything, avoid_spoiler]","[suks_suks, night_freddy]"


# 코드 참고자료

- https://towardsdatascience.com/clustering-sentence-embeddings-to-identify-intents-in-short-text-48d22d3bf02e
- https://www.kaggle.com/code/dardodel/steam-reviews-auto-topic-modeling-w-transformers
- https://jdjin3000.tistory.com/15
- https://www.kaggle.com/code/panks03/clustering-with-topic-modeling-using-lda/notebook